# Week 8 Seminar Answers

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as si
import yfinance as yf
import os

## Question 1

In [15]:
data = yf.download("^GSPC", start="2020-03-13", end="2021-03-13")

[*********************100%***********************]  1 of 1 completed


In [16]:
S = data['Adj Close'][-1]
print('The spot price is', round(S,2))

The spot price is 3943.34


## Question 2 

Step 1: Find the log return of Index

In [17]:
log_return = np.log(data['Adj Close'] / data['Adj Close'].shift(1))

Step 2: Calculate its standard deviation and annualised  it 

In [18]:
vol = np.sqrt(len(data)) * log_return.std()
print('The annualised volatility is', round(vol*100,2), '%')

The annualised volatility is 29.12 %


And the spot value for VIX is 

In [19]:
VIX = yf.download("^VIX", start="2020-03-13", end="2021-03-13")

[*********************100%***********************]  1 of 1 completed


In [20]:
vix = VIX['Adj Close'][-1]
print('The market volatility is', round(vix,2), '%')

The market volatility is 20.69 %


## Question 3

In [21]:
def euro_option_bs(S, K, T, r, vol, payoff):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #vol: volatility of underlying asset
    #payoff: call or put
    
    d1 = (np.log(S / K) + (r + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    if payoff == "call":
        option_value = S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
    elif payoff == "put":
        option_value = - S * si.norm.cdf(-d1, 0.0, 1.0) + K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0)
    
    return option_value

In [22]:
K = 4000
T = 9/12
r = 0.0164

In [23]:
call = euro_option_bs(S, K, T, r, vol, 'call')
print('The BS call price is', round(call,2))

The BS call price is 392.21


In [24]:
put = euro_option_bs(S, K, T, r, vol, 'put')
print('The BS put price is', round(put,2))

The BS put price is 399.97


## Question 4

In [25]:
SP500 = yf.Ticker("^GSPC")

In [26]:
opt = SP500.option_chain('2021-12-17')
call1 = opt.calls.lastPrice[opt1.calls.strike == K]
print('The Market call price is', float(call1.values))

NameError: name 'opt1' is not defined

In [ ]:
put1 = opt.puts.lastPrice[opt1.puts.strike == K]
print('The Market put price is', float(put1.values))

## Question 5

In [ ]:
call2 = euro_option_bs(S, K, T, r, vix*0.01, 'call')
print('The BS call price is', round(call2,2))

In [ ]:
put2 = euro_option_bs(S, K, T, r, vix*0.01, 'put')
print('The BS put price is', round(put2,2))

The results using the VIX is much better.

## Question 6

In [ ]:
K = np.linspace(3000,4000,11)
T = [1/52, 2/52, 1/12, 2/12, 1/4, 1/2, 1]

In [ ]:
call = np.zeros((len(T),len(K)))
for i in range(len(T)):
    for j in range(len(K)):
        call[i,j] = euro_option_bs(S, K[j], T[i], r, vix*0.01, 'call')

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

strike, ttm = np.meshgrid(K, T)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(10, 6))
ax = fig.gca(projection='3d')
surf = ax.plot_surface(strike, ttm, call, rstride=2, cstride=2, cmap=plt.cm.coolwarm, linewidth=0.5, antialiased=True)
ax.set_xlabel('Strike')
ax.set_ylabel('Time to expiry')
ax.set_zlabel('Call price')
fig.colorbar(surf, shrink=0.5, aspect=5);